# How do I access Cloud Data from my Local Machine in Python?

## Downloading data

When you have found the data you want to use, you have two options.  You can download the data to work locally, or access the data directly to work in the cloud. This second way of working is called "_Direct Cloud Access_" or simply, "_Direct Access_". 

This page covers downloading data from Earthdata Cloud to your local machine using the `earthaccess` library.

## earthaccess

We can use the [`earthaccess`](https://nsidc.github.io/earthaccess/) python library to grab the file URLs and then access them with the `xarray` library. 

In [1]:
#Import packages
import earthaccess
import xarray as xr

In [2]:
#Authentication with Earthdata Login
auth = earthaccess.login(strategy="netrc")

You're now authenticated with NASA Earthdata Login
Using token with expiration date: 10/30/2023
Using .netrc file for EDL


In [3]:
#Access land ice height from ATLAS/ICESat-2 V005 (10.5067/ATLAS/ATL06.005), searching for data over western Greenland coast over two weeks in July 2022. The data are provided as HDF5 granules (files) that span about 1/14th of an orbit.

results = earthaccess.search_data(short_name="ATL06",
                                  version="005",
                                  cloud_hosted=True,
                                  temporal = ("2022-07-17","2022-07-31"),
                                  bounding_box = (-51.96423,68.10554,-48.71969,70.70529))

Granules found: 5


In [11]:
files = earthaccess.download(results, "./local_folder")

downloaded_files = []
for f in files: 
    file = "local_folder/"+ f
    downloaded_files.append(file)

 Getting 5 granules, approx download size: 0.0 GB


SUBMITTING | :   0%|          | 0/5 [00:00<?, ?it/s]

File ATL06_20220718010029_03991603_005_01.h5 already downloaded
File ATL06_20220718134522_04071605_005_01.h5 already downloaded
File ATL06_20220722005209_04601603_005_01.h5 already downloaded
File ATL06_20220726004352_05211603_005_01.h5 already downloaded
File ATL06_20220722133704_04681605_005_01.h5 already downloaded


PROCESSING | :   0%|          | 0/5 [00:00<?, ?it/s]

COLLECTING | :   0%|          | 0/5 [00:00<?, ?it/s]

['local_folder/ATL06_20220718010029_03991603_005_01.h5',
 'local_folder/ATL06_20220718134522_04071605_005_01.h5',
 'local_folder/ATL06_20220722005209_04601603_005_01.h5',
 'local_folder/ATL06_20220722133704_04681605_005_01.h5',
 'local_folder/ATL06_20220726004352_05211603_005_01.h5']

In [12]:
#Use xarray to load the data as a multifile dataset for a single group in the HDF5 file, in this case land ice segments:
ds = xr.open_mfdataset(downloaded_files, group='/gt1l/land_ice_segments', engine='h5netcdf')
ds

<xarray.Dataset>
Dimensions:                (delta_time: 241711)
Coordinates:
  * delta_time             (delta_time) datetime64[ns] 2022-07-18T01:00:46.67...
    latitude               (delta_time) float64 dask.array<chunksize=(78325,), meta=np.ndarray>
    longitude              (delta_time) float64 dask.array<chunksize=(78325,), meta=np.ndarray>
Data variables:
    atl06_quality_summary  (delta_time) int8 dask.array<chunksize=(78325,), meta=np.ndarray>
    h_li                   (delta_time) float32 dask.array<chunksize=(78325,), meta=np.ndarray>
    h_li_sigma             (delta_time) float32 dask.array<chunksize=(78325,), meta=np.ndarray>
    segment_id             (delta_time) float64 dask.array<chunksize=(78325,), meta=np.ndarray>
    sigma_geo_h            (delta_time) float32 dask.array<chunksize=(78325,), meta=np.ndarray>
Attributes:
    Description:  The land_ice_height group contains the primary set of deriv...
    data_rate:    Data within this group are sparse.  Data values are provide...